In [69]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from xml.dom import minidom

In [70]:
img_dir='../../../../Data/VOCdevkit/VOC2012/JPEGImages/'
class_label_dir='../../../../Data/VOCdevkit/VOC2012/ImageSets/Main/'
anotation_dir='../../../../Data/VOCdevkit/VOC2012/Annotations/'
seg_class_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationClass/'
seg_obj_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationObject/'

## 找到图像对应的label 

In [71]:
categories=[]
for root,dirs,files in os.walk(class_label_dir):
    for file in files:
        if 'trainval' in file and '_' in file:
            categories.append(file.split('_')[0])

In [72]:
categories

['dog',
 'sofa',
 'bicycle',
 'aeroplane',
 'pottedplant',
 'car',
 'horse',
 'bird',
 'bottle',
 'cow',
 'cat',
 'tvmonitor',
 'person',
 'train',
 'boat',
 'diningtable',
 'sheep',
 'chair',
 'motorbike',
 'bus']

In [73]:
img_with_label={}
for i in range(len(categories)):
    cate=class_label_dir+categories[i]+'_trainval.txt'
    with open(cate,'r') as f:
        data=f.readlines()
        
        for line in data:
            items=line.split(' ')
            if items[0] in img_with_label.keys():
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1
            else:
                img_with_label[items[0]]=[-1 for _ in range(20)]
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1

# Create Dataframe for all imgs

In [74]:
imgs=[]
for root,dirs,files in os.walk(img_dir):
    for file in files:
        imgs.append(file.split('.')[0])

In [75]:
img_classes=[]
for img in imgs:
    if img in img_with_label.keys():
        img_classes.append(img_with_label[img])
    else:
        img_classes.append(np.nan)

In [76]:
img_bbox=[]
img_bbox_label=[]
for img in tqdm(imgs):
    if os.path.exists(anotation_dir+img+'.xml'):
        single_img_bbox=[]
        single_bbox_label=[]
        
        mydoc = minidom.parse(anotation_dir+img+'.xml')
        root=mydoc.documentElement
        
        items = root.getElementsByTagName('object')
        for item in items:
            single_bbox=[]
            name=item.getElementsByTagName('name')[0].firstChild.data
            bbox=item.getElementsByTagName('bndbox')[0]

            for b in bbox.childNodes:
                if b.nodeType==1:
                    single_bbox.append(b.firstChild.data)
                    
            single_img_bbox.append(single_bbox)
            single_bbox_label.append(name)
        
        img_bbox.append(single_img_bbox)
        img_bbox_label.append(single_bbox_label)
    else:
        img_bbox.append(np.nan)
        img_bbox_label.append(np.nan)

In [77]:
img_seg_class=[]
for img in tqdm(imgs):
    if os.path.exists(seg_class_dir+img+'.png'):
        img_seg_class.append(1)
    else:
        img_seg_class.append(np.nan)

In [78]:
img_seg_obj=[]
for img in tqdm(imgs):
    if os.path.exists(seg_obj_dir+img+'.png'):
        img_seg_obj.append(1)
    else:
        img_seg_obj.append(np.nan)

In [79]:
data_csv=pd.DataFrame({'img_name':imgs})

In [80]:
data_csv['img_class']=img_classes

In [81]:
data_csv['img_bbox']=img_bbox

In [82]:
data_csv['seg_class']=img_seg_class

In [83]:
data_csv['seg_obj']=img_seg_obj

In [84]:
data_csv['bbox_label']=img_bbox_label

In [85]:
data_csv.head()

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[[499, 1, 293, 19]]",NaN,NaN,[cat]
1,2012_003811,NaN,"[[56, 11, 277, 151]]",NaN,NaN,[person]
2,2007_007048,NaN,"[[227, 55, 500, 324], [82, 56, 245, 257], [76,...",1.0,1.0,"[sheep, sheep, bottle, person, person, person,..."
3,2012_001460,NaN,"[[230, 142, 229, 53]]",NaN,NaN,[person]
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[1, 228, 95, 343], [77, 198, 101, 266], [127,...",NaN,NaN,"[car, person, person, person, person]"


## delete len(img_bbox)!= img_class 

In [86]:
def return_one(arr):
    res=0
    for i in arr:
        if i==1:
            res+=1
    
    return res

In [88]:
del_index=[]
for index in data_csv.index:
    if type(data_csv.at[index,'img_class'])==type([1,2,3]) and type(data_csv.at[index,'bbox_label'])==type(['a']):
        if return_one(data_csv.at[index,'img_class'])!=len(set(data_csv.at[index,'bbox_label'])):
            del_index.append(index)

In [90]:
len(del_index)

0

In [91]:
data_csv.to_csv('data.csv',index=False)